In [1]:
import torch.nn            as nn
import numpy               as np
import matplotlib.pyplot   as plt
import torch.nn.functional as F
import networkx            as nx
import itertools
import torch

from os                   import path
from torch_geometric.data import Data
from pymatgen.core        import Structure
from torch.nn             import Linear

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Graph embedding

In [2]:
distance_threshold = 5
y = 1
cell, composition, concentration, positions = MPL.information_from_VASPfile('.',
                                                                            'POSCAR')

nodes, edges, attributes = MPL.graph_POSCAR_encoding(None,
                                                    cell,
                                                    composition,
                                                    concentration,
                                                    positions,
                                                    distance_threshold=distance_threshold
                                                   )

#attributes = attributes * 100
temp = Data(x=nodes,
            edge_index=edges.t().contiguous(),
            edge_attr=attributes.flatten(),
            y=torch.tensor([y], dtype=torch.float)
           )
temp

Data(x=[71, 5], edge_index=[2, 586], edge_attr=[586], y=[1])

In [3]:
def get_n_connected(idx_0, cartesian_positions, edge_indexes, edge_attributes):
    n_connected = 0
    idx_connected = []
    for idx_t in list(cartesian_positions.keys()):
        if get_distance_attribute(idx_0, idx_t, edge_indexes, edge_attributes) is not None:
            n_connected += 1
            idx_connected.append(idx_t)
    
    return n_connected, idx_connected


def allocate_atom_n(P1, P2, P3, r1, r2, r3):
    """
    Trilateration function to find the coordinates of the unknown point given three reference points
    and their corresponding distances to the unknown point.

    Args:
        P1 (numpy.ndarray): Coordinates of the first reference point.
        P2 (numpy.ndarray): Coordinates of the second reference point.
        P3 (numpy.ndarray): Coordinates of the third reference point.
        r1 (float): Distance from the unknown point to the first reference point.
        r2 (float): Distance from the unknown point to the second reference point.
        r3 (float): Distance from the unknown point to the third reference point.

    Returns:
        Tuple[numpy.ndarray, numpy.ndarray]: Coordinates of the unknown point in two possible solutions.
    """

    # Calculate vector representations of the reference points
    p1 = np.array([0, 0, 0])
    p2 = np.array([P2[0] - P1[0], P2[1] - P1[1], P2[2] - P1[2]])
    p3 = np.array([P3[0] - P1[0], P3[1] - P1[1], P3[2] - P1[2]])

    # Calculate vectors representing the directions from the first reference point
    # to the second and third reference points
    v1 = p2 - p1
    v2 = p3 - p1

    # Calculate unit vectors in the Xn, Yn, and Zn directions
    Xn = v1 / np.linalg.norm(v1)
    tmp = np.cross(v1, v2)
    Zn = tmp / np.linalg.norm(tmp)
    Yn = np.cross(Xn, Zn)

    # Calculate dot products and distances
    i = np.dot(Xn, v2)
    d = np.dot(Xn, v1)
    j = np.dot(Yn, v2)

    # Calculate coordinates of the unknown point in two possible solutions
    X = (np.power(r1, 2) - np.power(r2, 2) + np.power(d, 2)) / (2 * d)
    Y = (np.power(r1, 2) - np.power(r3, 2) + np.power(i, 2) + np.power(j, 2)) / (2 * j) - X * i / j
    Z1 = np.sqrt(max(0, np.power(r1, 2) - np.power(X, 2) - np.power(Y, 2)))

    # Calculate the coordinates of the unknown point for both solutions
    K1 = P1 + X * Xn + Y * Yn + Z1 * Zn
    return K1


def get_distance_attribute(index0, index1, edge_indexes, edge_attributes):
    """Get the distance attribute between two nodes with given indices.

    Args:
        index0          (int):        Index of the first node.
        index1          (int):        Index of the second node.
        edge_indexes    (np.ndarray): Array containing indices of connected nodes for each edge.
        edge_attributes (np.ndarray): Array containing attributes corresponding to each edge.

    Returns:
        float: The distance attribute between the two nodes.
        False if index0, index1 not linked.
    """
    
    # Create a mask to find matching edges
    mask_direct  = (edge_indexes[0] == index0) & (edge_indexes[1] == index1)
    mask_reverse = (edge_indexes[0] == index1) & (edge_indexes[1] == index0)
    
    # Check if any edge satisfies the conditions
    matching_edge_indices = np.where(mask_direct | mask_reverse)[0]
    
    if len(matching_edge_indices) == 0:
        return None  # The pair is not linked
    
    # Get the distance attribute from the first matching edge
    distance_attribute = edge_attributes[matching_edge_indices[0]]
    
    return distance_attribute


def find_initial_basis(total_particles, edge_indexes, edge_attributes):
    for idx_0 in range(total_particles):
        for idx_1 in np.arange(idx_0+1, total_particles):
            for idx_2 in np.arange(idx_1+1, total_particles):
                condition_01 = (get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes) is not None)
                condition_12 = (get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes) is not None)
                condition_02 = (get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes) is not None)
                if condition_01 and condition_12 and condition_02:
                    return idx_0, idx_1, idx_2


def find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes):
    for i in range(n_connected):
        for j in np.arange(i+1, n_connected):
            for k in np.arange(j+1, n_connected):
                idx_0 = idx_connected[i]
                idx_1 = idx_connected[j]
                idx_2 = idx_connected[k]

                r0 = cartesian_positions[idx_0]
                r1 = cartesian_positions[idx_1]
                r2 = cartesian_positions[idx_2]
                
                # Get necessary distances
                d_0n = get_distance_attribute(idx_0, idx, edge_indexes, edge_attributes)
                d_1n = get_distance_attribute(idx_1, idx, edge_indexes, edge_attributes)
                d_2n = get_distance_attribute(idx_2, idx, edge_indexes, edge_attributes)
                
                temp_position = allocate_atom_n(r0, r1, r2, d_0n, d_1n, d_2n)
                
                if temp_position is not None:
                    return temp_position

In [4]:
r0 = np.array([0.1146855891840775, 0.1364174984341713, 0.2949503084492082])
r1 = np.array([0.6146855891840775, 0.1364174984341713, 0.2949503084492082])
r2 = np.array([0.1146855891840775, 0.6364174984341713, 0.2949503084492082])
d  = np.array([0.6146855891840775, 0.6364174984341713, 0.2949503084492082])

# Distances
d_0n = np.linalg.norm([r0 - d])
d_1n = np.linalg.norm([r1 - d])
d_2n = np.linalg.norm([r2 - d])

allocate_atom_n(r0, r1, r2, d_0n, d_1n, d_2n)

array([0.61468559, 0.6364175 , 0.29495032])

In [12]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected
idx_0, idx_1, idx_2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Get necessary distances
d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

# Reference the first three atoms
x2 = (np.power(d_01, 2) + np.power(d_02, 2) - np.power(d_12, 2)) / (2 * d_01)
y2 = np.sqrt(np.power(d_02, 2) - np.power(x2, 2))

# Impose three particles at the beginning
cartesian_positions = {
    idx_0: np.array([0,    0,  0]),
    idx_1: np.array([d_01, 0,  0]),
    idx_2: np.array([x2,   y2, 0])
}

print(np.linalg.norm(cartesian_positions[idx_1] - cartesian_positions[idx_0]), d_01)
print(np.linalg.norm(cartesian_positions[idx_2] - cartesian_positions[idx_0]), d_02)
print(np.linalg.norm(cartesian_positions[idx_1] - cartesian_positions[idx_2]), d_12)

4.183528900146484 4.183529
4.217043399810791 4.2170434
4.213690280914307 4.2136903


In [6]:
# Extract indexes and attributes from the graph
edge_indexes    = edges.T.detach().cpu().numpy().copy()
edge_attributes = attributes.detach().cpu().numpy().copy().flatten()

# Define the number of atoms in the graph
total_particles = len(nodes)

# Select three initial particles which are interconnected
idx_0, idx_1, idx_2 = find_initial_basis(total_particles, edge_indexes, edge_attributes)

# Get necessary distances
d_01 = get_distance_attribute(idx_0, idx_1, edge_indexes, edge_attributes)
d_02 = get_distance_attribute(idx_0, idx_2, edge_indexes, edge_attributes)
d_12 = get_distance_attribute(idx_1, idx_2, edge_indexes, edge_attributes)

# Reference the first three atoms
x2 = (np.power(d_01, 2) + np.power(d_02, 2) - np.power(d_12, 2)) / (2 * d_01)
y2 = np.sqrt(np.power(d_02, 2) - np.power(x2, 2))

# Impose three particles at the beginning
cartesian_positions = {
    idx_0: np.array([0,    0,  0]),
    idx_1: np.array([d_01, 0,  0]),
    idx_2: np.array([x2,   y2, 0])
}

all_idxs = np.delete(np.arange(total_particles),
                     [idx_0, idx_1, idx_2])

# Initialized to 3 for three connections
n_connected_list = all_idxs.tolist()
idx_0, idx_1, idx_2

(0, 2, 4)

In [7]:
while len(n_connected_list):  # Goes until all particles have been studied
    # Using a first-one-first-out approach
    idx = n_connected_list[0]

    n_connected, idx_connected = get_n_connected(idx, cartesian_positions, edge_indexes, edge_attributes)
    print()
    print(idx)
    print(n_connected_list)

    # Remove idx from connected_dict
    n_connected_list.remove(idx)
    
    # Set idx in cartesian_positions or else add it to n_connected_list
    if n_connected >= 3:
        # Extract the cartesian coordinates of idx
        temp_position = find_valid_reference(n_connected, idx_connected, edge_indexes, edge_attributes)

        # Check if there are enough particles able to locate idx (images make this step more difficult
        if temp_position is not None:
            # Generate temporal dictionary with the cartesian coordinates
            temp_dict = {
                idx: temp_position
            }

            # Update general dictionary with cartesian coordinates
            cartesian_positions.update(temp_dict)
        else:
            # n_connected_list is updated adding idx where it belongs to
            n_connected_list.append(idx)
    else:
        # n_connected_list is updated adding idx where it belongs to
        n_connected_list.append(idx)


1
[1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]

3
[3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1]

5
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 1, 3]

6
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 5

In [10]:
for i in range(total_particles):
    print(cartesian_positions[i])

[0 0 0]
[ 2.65761349 -0.18201158 -0.8668359 ]
[4.1835289 0.        0.       ]
[ 2.84683699  0.36970009 -3.57879778]
[2.09514309 3.65975825 0.        ]
[ 4.27724714  2.59567497 -1.13398334]
[ 1.86472653 -1.81925621  2.89032225]
[ 5.42985916 -0.12842499 -0.42251742]
[4.89286347 2.65702585 2.66661918]
[ 1.40620169 -1.99763492 -0.97943489]
[2.5473926  1.03308778 2.82690778]
[ 2.70700796 -0.21618153  1.45563375]
[ 3.2407954  -0.97835469  4.01959904]
[-0.83263642 -1.47221977  4.65744883]
[ 4.22861758 -3.60090562  3.42702671]
[0.06895804 4.24704717 0.4404175 ]
[ 1.67380593  3.75176476 -1.23756446]
[ 1.65377068  2.89995495 -0.69200652]
[1.12837718 2.32771174 1.43825998]
[ 6.63850562 -0.80044509  2.04272072]
[0.8610142  2.29324425 1.69113876]
[ 1.85938498 -0.25617315  1.16511881]
[-1.7610207   1.57886395  1.95874592]
[3.0062919  0.41623443 0.3425502 ]
[ 2.06915974 -1.74335847  0.32024349]
[3.06482848 0.40447939 1.0299666 ]
[4.6452001  2.7265712  0.10601612]
[ 3.8276321   0.04865075 -2.17562734]